# Introction to Tempo on Kubernetes

This notebook will walk you through an end-to-end example deploying a Tempo pipeline deployed to Kubernetes.

## Prerequisites

This notebooks needs to be run in the `tempo-examples` conda environment defined below. Create from project root folder:

```bash
conda env create --name tempo-examples --file conda/tempo-examples.yaml
```

## Architecture

We will show two Iris dataset prediction models combined with service orchestration that shows some arbitrary python code to control predictions from the two models.

![architecture](architecture.png)

In [1]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Train Iris Models

We will train:

  * A sklearn logistic regression model
  * A xgboost model

In [2]:
!mkdir -p artifacts/sklearn
!mkdir -p artifacts/xgboost

In [3]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
import joblib
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
y = iris.target
logreg = LogisticRegression(C=1e5)
logreg.fit(X, y)
logreg.predict_proba(X[0:1])
with open("./artifacts/sklearn/model.joblib","wb") as f:
    joblib.dump(logreg, f)

In [4]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(X,y)
clf.save_model("./artifacts/xgboost/model.bst")

[12:21:26] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/rskolasinski/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## Defining pipeline

The first step will be to define our custom pipeline.
This pipeline will access 2 models, stored remotely. 

In [5]:
import os
import numpy as np
from typing import Tuple
from tempo.serve.metadata import ModelFramework, KubernetesOptions, RuntimeOptions
from tempo.serve.model import Model
from tempo.serve.pipeline import PipelineModels
from tempo.seldon.protocol import SeldonProtocol
from tempo.seldon.docker import SeldonDockerRuntime
from tempo.kfserving.protocol import KFServingV2Protocol, KFServingV1Protocol
from tempo.kfserving.k8s import KFServingKubernetesRuntime
from tempo.serve.utils import pipeline, predictmethod
from tempo.seldon.k8s import SeldonKubernetesRuntime
from tempo.serve.utils import pipeline
from tempo.serve.loader import upload, save

import logging
logging.basicConfig(level=logging.INFO)


SKLEARN_FOLDER = f"{os.getcwd()}/artifacts/sklearn"
XGBOOST_FOLDER = f"{os.getcwd()}/artifacts/xgboost"
PIPELINE_ARTIFACTS_FOLDER = f"{os.getcwd()}/artifacts/classifier"

runtimeOptions=RuntimeOptions(  
    k8s_options=KubernetesOptions( 
        defaultRuntime="tempo.kfserving.KFServingKubernetesRuntime",
        namespace="production",
        serviceAccountName="kf-tempo"
    )
)


sklearn_model = Model(
    name="test-iris-sklearn",
    platform=ModelFramework.SKLearn,
    protocol=KFServingV2Protocol(),
    runtime_options=runtimeOptions,
    local_folder=SKLEARN_FOLDER,
    uri="s3://tempo/basic/sklearn",
    inputs=np.ndarray,
    outputs=np.ndarray
)

xgboost_model = Model(
    name="test-iris-xgboost",
    platform=ModelFramework.XGBoost,
    protocol=KFServingV2Protocol(),
    runtime_options=runtimeOptions,
    local_folder=XGBOOST_FOLDER,
    uri="s3://tempo/basic/xgboost",
    inputs=np.ndarray,
    outputs=np.ndarray    
)

@pipeline(
    name="classifier",
    uri="s3://tempo/basic/pipeline",
    local_folder=PIPELINE_ARTIFACTS_FOLDER,
    runtime_options=runtimeOptions,
    models=PipelineModels(sklearn=sklearn_model, xgboost=xgboost_model)
)
def classifier(payload: np.ndarray) -> Tuple[np.ndarray,str]:
    res1 = classifier.models.sklearn(input=payload)
    print(res1)
    if res1[0] == 1:
        return res1,"sklearn prediction"
    else:
        return classifier.models.xgboost(input=payload),"xgboost prediction"

### Saving artifacts

We provide a conda yaml in out `local_folder` which tempo will use as the runtime environment to save. If this file was not there it would save the current conda environment. One can also provide a named conda environment with `conda_env` in the decorator.

In [6]:
import sys
import os
PYTHON_VERSION = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
TEMPO_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))

In [7]:
%%writetemplate artifacts/classifier/conda.yaml
name: tempo
channels:
  - defaults
dependencies:
  - python={PYTHON_VERSION}
  - pip:
    - mlops-tempo @ file://{TEMPO_DIR}
    - mlserver==0.3.1.dev7

In [8]:
save(classifier, save_env=True)

INFO:tempo:Saving environment
INFO:tempo:Saving tempo model to /home/rskolasinski/work/tempo/docs/examples/kfserving/artifacts/classifier/model.pickle
INFO:tempo:Using found conda.yaml
INFO:tempo:Creating conda env with: conda env create --name tempo-4ba01731-d002-4325-af4c-e27a844f7b1c --file /tmp/tmpr66ieu3k.yml
INFO:tempo:packing conda environment from tempo-4ba01731-d002-4325-af4c-e27a844f7b1c


Packing environment at '/home/rskolasinski/miniconda3/envs/tempo-4ba01731-d002-4325-af4c-e27a844f7b1c' to '/home/rskolasinski/work/tempo/docs/examples/kfserving/artifacts/classifier/environment.tar.gz'
[########################################] | 100% Completed | 10.7s


INFO:tempo:Removing conda env with: conda remove --name tempo-4ba01731-d002-4325-af4c-e27a844f7b1c --all --yes


## Deploying pipeline to K8s

The next step, will be to deploy our pipeline to Kubernetes.
We will divide this process into 3 sub-steps:

1. Save our artifacts and environment
2. Upload to remote storage
3. Deploy resources

### Setup Namespace with Minio Secret

In [9]:
!kubectl create namespace production

Error from server (AlreadyExists): namespaces "production" already exists


In [10]:
%%writefile auth.yaml
---
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
  annotations:
     serving.kubeflow.org/s3-endpoint: minio.minio-system.svc.cluster.local:9000 # replace with your s3 endpoint
     serving.kubeflow.org/s3-usehttps: "0" # by default 1, for testing with minio you need to set to 0
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: minioadmin
  AWS_SECRET_ACCESS_KEY: minioadmin
---
apiVersion: v1
kind: ServiceAccount
metadata:
  name: kf-tempo
secrets:
- name: minio-secret
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: kf-tempo
rules:
  - apiGroups:
      - machinelearning.seldon.io
    resources:
      - seldondeployments/status
    verbs:
      - get
  - apiGroups:
      - serving.kubeflow.org
    resources:
      - inferenceservices/status
    verbs:
      - get
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: tempo-pipeline-rolebinding
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: kf-tempo
subjects:
  - kind: ServiceAccount
    name: kf-tempo

Overwriting auth.yaml


In [11]:
!kubectl apply -f auth.yaml -n production

secret/minio-secret configured
serviceaccount/kf-tempo configured
role.rbac.authorization.k8s.io/kf-tempo unchanged
rolebinding.rbac.authorization.k8s.io/tempo-pipeline-rolebinding unchanged


### Uploading artifacts

In [12]:
MINIO_IP=!kubectl get svc minio -n minio-system -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MINIO_IP=MINIO_IP[0]

In [13]:
%%writetemplate rclone.conf
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://{MINIO_IP}:9000

In [14]:
import os
from tempo.conf import settings
settings.rclone_cfg = os.getcwd() + "/rclone.conf"

In [15]:
upload(sklearn_model)
upload(xgboost_model)
upload(classifier)

INFO:tempo:Uploading /home/rskolasinski/work/tempo/docs/examples/kfserving/artifacts/sklearn to s3://tempo/basic/sklearn
INFO:tempo:Uploading /home/rskolasinski/work/tempo/docs/examples/kfserving/artifacts/xgboost to s3://tempo/basic/xgboost
INFO:tempo:Uploading /home/rskolasinski/work/tempo/docs/examples/kfserving/artifacts/classifier to s3://tempo/basic/pipeline


### Deploy

In [23]:
k8s_runtime = KFServingKubernetesRuntime()
k8s_runtime.deploy(classifier)
k8s_runtime.wait_ready(classifier)

### Sending requests

Lastly, we can now send requests to our deployed pipeline.
For this, we will leverage the `remote()` method, which will interact without our deployed pipeline (as opposed to executing our pipeline's code locally).

In [17]:
from tempo.utils import tempo_settings
tempo_settings.remote_kubernetes(True)

In [18]:
classifier(payload=np.array([[1, 2, 3, 4]]))

[2.]


(array([[0.00847207, 0.03168794, 0.95984   ]], dtype=float32),
 'xgboost prediction')

In [19]:
classifier.remote(payload=np.array([[1, 2, 3, 4]]))

ValueError: ('Bad return code', 500, '  File "/opt/mlserver/envs/environment/lib/python3.7/site-packages/starlette/middleware/errors.py", line 159, in __call__\n    await self.app(scope, receive, _send)\n  File "/opt/mlserver/envs/environment/lib/python3.7/site-packages/starlette/exceptions.py", line 82, in __call__\n    raise exc from None\n  File "/opt/mlserver/envs/environment/lib/python3.7/site-packages/starlette/exceptions.py", line 71, in __call__\n    await self.app(scope, receive, sender)\n  File "/opt/mlserver/envs/environment/lib/python3.7/site-packages/starlette/routing.py", line 566, in __call__\n    await route.handle(scope, receive, send)\n  File "/opt/mlserver/envs/environment/lib/python3.7/site-packages/starlette/routing.py", line 227, in handle\n    await self.app(scope, receive, send)\n  File "/opt/mlserver/envs/environment/lib/python3.7/site-packages/starlette/routing.py", line 41, in app\n    response = await func(request)\n  File "/opt/mlserver/envs/environment/lib

In [ ]:
classifier.remote(payload=np.array([[5.964,4.006,2.081,1.031]]))

### Undeploy pipeline

In [22]:
k8s_runtime.undeploy(classifier)

INFO:tempo:Undeploying classifier
INFO:tempo:Undeploying test-iris-sklearn
INFO:tempo:Undeploying test-iris-xgboost
